In [1]:
import torch
from torch import nn
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
data_name = "rte"
# model_name = f"./lora_trained/lora-t5-{data_name}/"
model_name = 'google/flan-t5-base'

print(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer = T5Tokenizer.from_pretrained(model_name)

google/flan-t5-base


### For rte model we have 144 lora matrix. of whoche 72 for query and 72 for value.
For each consists of 36 matrix A and 36 matrix B.
All lora matrix is of shape [768,8] and [8,768]. may need to reshape these matrix to get (d,r) and (r,d)

In [3]:
from peft.tuners.lora import LoraLayer

def extract_lora_matrices(model, save_path):
    lora_matrices = {}
    
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            lora_A = module.lora_A['default'].weight.data.cpu()
            lora_B = module.lora_B['default'].weight.data.cpu()
            lora_matrices[f"{name}_lora_A"] = lora_A
            lora_matrices[f"{name}_lora_B"] = lora_B

    torch.save(lora_matrices, save_path)
    print(f"LoRA matrices saved to {save_path}")

save_path=f'../MoLE/lora_matrices/{data_name}_lora_matrices.pth'
# extract_lora_matrices(model, save_path)


In [4]:


AB = torch.load(save_path)
type(AB)

/tmp/ipykernel_644200/1856948567.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  AB = torch.load(save_path)


dict

In [5]:
len(model.decoder.block)

12

In [6]:
count = 0
for name, param in model.named_parameters():
    # if "q.lora_A" in name:
    #     count+=1
    # if  not 'lora' in name: 
    count+=1
    # print(name)
    # print(param.shape)

print(count)

282


In [7]:
model.encoder.block = model.encoder.block[:3]
model.decoder.block = model.decoder.block[:3]
# Print the total number of layers after modification
print(f"Total number of layers after modification: {len(model.encoder.block)}")

Total number of layers after modification: 3


In [46]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

input_text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(input_text, return_tensors='pt')

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

embedding_output = model.encoder.embed_tokens(input_ids)

encoder_layer = model.encoder.block[0]


self_attention_output = encoder_layer.layer[0](
    embedding_output,
    attention_mask=attention_mask, 
)[0]  

feed_forward_output = encoder_layer.layer[1](self_attention_output)[0]

/home/rajeshupadhayaya/miniconda3/envs/seismic/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
feed_forward_output.shape

torch.Size([13, 768])

In [39]:
inputs['input_ids']

tensor([[   37,  1704,  4216,     3, 20400,  4418,     7,   147,     8, 19743,
          1782,     5,     1]])

In [45]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [36]:
m.last_hidden_state.shape

torch.Size([1, 13, 768])

In [32]:
inputs.input_ids.shape

torch.Size([1, 13])

In [ ]:
k